#Import Libraries

In [1]:

__author__ = 'aqeel'
'''Train and evaluate a simple MLP on the Souq.com Reviews newswire topic classification task.
GPU run command:
    THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python examples/NNClassifiyReviews.py
CPU run command:
    python examples/NNClassifiyReviews.py
'''
import numpy as np
np.random.seed(1337)
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Merge, Dropout, RepeatVector,MaxoutDense,Activation
from keras.layers import recurrent
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.objectives import mse as MMSE
import theano
import theano.tensor as K
import math
import random
random.seed(1337)

Using Theano backend.


#Read The Data
here we read the data and orgainze it in a file to be ready

In [3]:
mydata = pd.read_csv('All_astronomy_outliered.csv',sep=',',quotechar='"')

MORE_THAN_MEAN_BY= 0
mydata['a_words'] = map(lambda x:x.split(),mydata['body'])
mydata['q_words'] = map(lambda x:x.split(),mydata['q_body'])
mydata['q_bdlen'] = map(lambda x:len(x),mydata['q_words'])
mydata['a_bdlen'] = map(lambda x:len(x),mydata['a_words'])
Q_MAX = max(mydata['q_bdlen'])
A_MAX = max(mydata['a_bdlen'])
Q_MEAN = int(math.floor(np.mean(mydata['q_bdlen']))+MORE_THAN_MEAN_BY)
A_MEAN=int(math.floor(np.mean(mydata['a_bdlen']))+MORE_THAN_MEAN_BY)
print min(mydata['q_bdlen']),Q_MAX,Q_MEAN-MORE_THAN_MEAN_BY
print min(mydata['a_bdlen']),A_MAX,A_MEAN-MORE_THAN_MEAN_BY
#Q_MEAN = 38 #median
#A_MEAN = 86 #median
def GetDataW(splitper=0.2):
    global mydata
    splitper = int(math.floor(splitper * len(mydata)) + 1)
    #Shuffle the data
    mydata = mydata.iloc[np.random.permutation(len(mydata))]
    mydata[:splitper].to_csv('test.csv')
    return mydata['q_words'][splitper:].tolist(),mydata['a_words'][splitper:].tolist(),    mydata['score'][splitper:].tolist(),mydata['q_words'][:splitper].tolist(),    mydata['a_words'][:splitper].tolist(),mydata['score'][:splitper].tolist()

print ('Loading Data....')
Qx_trn,Ax_trn,y_trn,Qx_test,Ax_test,y_test= GetDataW(0.2)
print ('Data Loaded')

All_Vocabulary={}
for i in Qx_trn+Ax_trn+Qx_test+Ax_test:
    for element in i:
        All_Vocabulary[element]=True

All_Vocabulary= All_Vocabulary.keys()
print len(All_Vocabulary)

vocab_size = len(All_Vocabulary)+1
word_idx = dict((c, i + 1) for i, c in enumerate(All_Vocabulary))

def Vectorize():
    global Qx_trn,Ax_trn,Qx_test,Ax_test,y_trn,y_test
    
    for i in range (0,len(Qx_trn)):
        Qx_trn[i] = [word_idx[l] for l in Qx_trn[i]]
    Qx_trn= pad_sequences(Qx_trn,Q_MEAN)
    
    for i in range (0,len(Ax_trn)):
        Ax_trn[i] = [word_idx[l] for l in Ax_trn[i]]
    Ax_trn =pad_sequences(Ax_trn,A_MEAN)
    
    for i in range (0,len(Qx_test)):
        Qx_test[i] = [word_idx[l] for l in Qx_test[i]]
    Qx_test= pad_sequences(Qx_test,Q_MEAN)
    
    for i in range (0,len(Ax_test)):
        Ax_test[i] = [word_idx[l] for l in Ax_test[i]]
    Ax_test = pad_sequences(Ax_test,A_MEAN)
    y_trn = np.array(y_trn)
    y_test = np.array(y_test)

Vectorize()

3 3095 48
3 1858 112
Loading Data....
Data Loaded
38745


In [4]:
print('All Vocabulary  = {}'.format(vocab_size))
print('Train Questions.shape = {},Test Questions.shape{}'.format(Qx_trn.shape,Qx_test.shape))
print('Train Answers.shape = {},Test Answers.shape{}'.format(Ax_trn.shape,Ax_test.shape))
print('Train_Y.shape = {},Test_Y.shape{}'.format(y_trn.shape,y_test.shape))
print('Questions Max Length, Answers Max Length = {}, {}'.format(Q_MAX, A_MAX))

All Vocabulary  = 38746
Train Questions.shape = (3567, 48),Test Questions.shape(892, 48)
Train Answers.shape = (3567, 112),Test Answers.shape(892, 112)
Train_Y.shape = (3567,),Test_Y.shape(892,)
Questions Max Length, Answers Max Length = 3095, 1858


In [5]:
RNN = recurrent.LSTM
EMBED_HIDDEN_SIZE = 100
LSTM_HIDDEN_SIZE = 100
#1-inf
BATCH_SIZE = 32
#1-inf
EPOCHS = 2
#Done
#SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax
theoptimizer = 'Adam'
#DONE
#0.1-0.9
thedropout =0.5
#DONE
#softmax,softplus,relu,tanh,sigmoid,hard_sigmoid,linear,
FirstActivation = 'relu'
SecondActivation='softmax'
#DONE
#mean_squared_error / mse,root_mean_squared_error / rmse,mean_absolute_error / mae,mean_absolute_percentage_error / mape
#mean_squared_logarithmic_error / msle,squared_hinge, hinge,binary_crossentropy: Also known as logloss,categorical_crossentropy: Also known as multiclass logloss. Note: using this objective requires that your labels are binary arrays of shape (nb_samples, nb_classes).
#poisson: mean of (predictions - targets * log(predictions))# cosine_proximity: the opposite (negative) of the mean cosine proximity between predictions and targets.
theloss='NMSE'
#DONE
#======================
results = open('results.txt','a')
results.write('\nBatchSize:{},EPOCH:{},Optimizer:{},Dropout:{},1stActivation:{},2ndActivation:{},theloss:{},QVectorLength:{},AVectorLength:{},HIDDENSIZE:{}'\
.format(BATCH_SIZE,EPOCHS,theoptimizer,thedropout,FirstActivation,SecondActivation,theloss,Q_MEAN,A_MEAN,EMBED_HIDDEN_SIZE))


In [6]:
print('Build model...')

Questions = Sequential()
Questions.add(Embedding(vocab_size, EMBED_HIDDEN_SIZE, input_length=Q_MEAN, mask_zero=True))
Questions.add(Dropout(thedropout))
Questions.add(RNN(LSTM_HIDDEN_SIZE, return_sequences=False))

Answers = Sequential()
Answers.add(Embedding(vocab_size, EMBED_HIDDEN_SIZE, input_length=A_MEAN,mask_zero=True))
Answers.add(Dropout(thedropout))
Answers.add(RNN(LSTM_HIDDEN_SIZE, return_sequences=False))

print Questions.output_shape
print Answers.output_shape
m = Merge([Questions, Answers], mode='concat')
print m.output_shape

def NMSE(y_true, y_pred):
    nmse = K.mean(K.square(y_pred-y_true)/(K.mean(y_pred)*K.mean(y_true)))
    results.write('\n MSE IS:= {}'.format(MMSE(y_true,y_pred)))
    return nmse

model = Sequential()
model.add(Merge([Questions, Answers], mode='concat'))
model.add(Dropout(0.3))
model.add(Dense(100, activation='linear'))
model.add(Dense(100, activation='linear'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer=theoptimizer,
              loss=NMSE)

Build model...
(None, 100)
(None, 100)
(None, 200)


In [7]:
Questions.summary()

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
embedding_1 (Embedding)            (None, 48, 100)     3874600     embedding_input_1[0][0]          
____________________________________________________________________________________________________
dropout_1 (Dropout)                (None, 48, 100)     0           embedding_1[0][0]                
____________________________________________________________________________________________________
lstm_1 (LSTM)                      (None, 100)         80400       dropout_1[0][0]                  
Total params: 3955000
____________________________________________________________________________________________________


In [8]:
Answers.summary()

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
embedding_2 (Embedding)            (None, 112, 100)    3874600     embedding_input_2[0][0]          
____________________________________________________________________________________________________
dropout_2 (Dropout)                (None, 112, 100)    0           embedding_2[0][0]                
____________________________________________________________________________________________________
lstm_2 (LSTM)                      (None, 100)         80400       dropout_2[0][0]                  
Total params: 3955000
____________________________________________________________________________________________________


In [9]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
embedding_1 (Embedding)            (None, 48, 100)     3874600                                      
____________________________________________________________________________________________________
dropout_1 (Dropout)                (None, 48, 100)     0                                            
____________________________________________________________________________________________________
lstm_1 (LSTM)                      (None, 100)         80400                                        
____________________________________________________________________________________________________
embedding_2 (Embedding)            (None, 112, 100)    3874600                                      
___________________________________________________________________________________________

In [ ]:
print('Training')
hist = model.fit([Qx_trn, Ax_trn], y_trn, batch_size=BATCH_SIZE, nb_epoch=EPOCHS, validation_split=0.05)
losst= model.evaluate([Qx_trn, Ax_trn], y_trn, batch_size=BATCH_SIZE)
results.write('\n Train loss= {}'.format(losst))
loss= model.evaluate([Qx_test, Ax_test], y_test, batch_size=BATCH_SIZE)
results.write('\n Test loss = {}'.format(loss))
results.close()
prediction_Y= model.predict([Qx_test, Ax_test],batch_size=BATCH_SIZE)
with open('RNNOUTPUT.csv','w') as output:
    output.write('\"id\",\"relevance\"\n')
    for i in range(len(prediction_Y)):
        output.write('{}'.format(prediction_Y[i][0])+'\n')

# Sample Output 
Copied From the Server 
The Folloiwng parameters are the same in all experiments change :

theloss:NMSE,QVectorLength:50,AVectorLength:117, Optimizer:Adam

EPOCH:2,HIDDENSIZE:4

Train loss = 1.01326246647
Test loss  = 1.0573939355

EPOCH:2,HIDDENSIZE:10

Train loss / Train accuracy = 8.17936526422 / 0.188673955764
Test loss / test accuracy = 9.2275516912 / 0.177130044576

EPOCH:50,HIDDENSIZE:4

Train loss= 0.105274160135
Test loss = 1.05210174493

EPOCH:50,HIDDENSIZE:20

Train loss= 0.0716473343144
Test loss = 0.967259530797

EPOCH:50,HIDDENSIZE:100

Train loss= 0.0706326634774
Test loss = 1.13603488613

EPOCH:2,HIDDENSIZE:100

Train loss= 0.73422428308
Test loss = 0.813926397418

**The MSE For this value is :9.695838**
**Best Result with MSE Cost Function:**
EPOCH:1,theloss:mse,,HIDDENSIZE:4
 Train loss  = 7.92233450552
 Test loss = 8.85901421175
